## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config3 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Satka,RU,55.0425,59.0400,33.60,84,100,13.62,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,50.11,57,75,33.38,broken clouds
2,2,Cururupu,BR,-1.8283,-44.8683,83.43,66,19,5.93,few clouds
3,3,Belmonte,BR,-15.8631,-38.8828,77.50,81,81,9.71,broken clouds
4,4,Qaqortoq,GL,60.7167,-46.0333,43.30,47,0,0.78,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Cururupu,BR,-1.8283,-44.8683,83.43,66,19,5.93,few clouds
3,3,Belmonte,BR,-15.8631,-38.8828,77.50,81,81,9.71,broken clouds
10,10,Kapaa,US,22.0752,-159.3190,82.38,74,40,4.00,scattered clouds
11,11,Castro,BR,-24.7911,-50.0119,82.65,41,16,2.13,few clouds
16,16,Coro,VE,11.4045,-69.6734,89.53,63,22,21.90,few clouds
17,17,Sao Filipe,CV,14.8961,-24.4956,80.78,76,10,11.18,clear sky
20,20,Rikitea,PF,-23.1203,-134.9692,73.76,71,29,22.48,scattered clouds
21,21,Ler,SS,8.3018,30.1418,77.70,71,95,2.98,overcast clouds
23,23,Albany,US,42.6001,-73.9662,74.35,70,91,7.54,overcast clouds
25,25,Alta Floresta,BR,-9.8756,-56.0861,85.82,62,20,5.75,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                322
City                   322
Country                321
Lat                    322
Lng                    322
Max Temp               322
Humidity               322
Cloudiness             322
Wind Speed             322
Current Description    322
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Cururupu,BR,-1.8283,-44.8683,83.43,66,19,5.93,few clouds
3,3,Belmonte,BR,-15.8631,-38.8828,77.50,81,81,9.71,broken clouds
10,10,Kapaa,US,22.0752,-159.3190,82.38,74,40,4.00,scattered clouds
11,11,Castro,BR,-24.7911,-50.0119,82.65,41,16,2.13,few clouds
16,16,Coro,VE,11.4045,-69.6734,89.53,63,22,21.90,few clouds


In [9]:
clean_travel_cities.count()

City_ID                321
City                   321
Country                321
Lat                    321
Lng                    321
Max Temp               321
Humidity               321
Cloudiness             321
Wind Speed             321
Current Description    321
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cururupu,BR,83.43,few clouds,-1.8283,-44.8683,
3,Belmonte,BR,77.50,broken clouds,-15.8631,-38.8828,
10,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,
11,Castro,BR,82.65,few clouds,-24.7911,-50.0119,
16,Coro,VE,89.53,few clouds,11.4045,-69.6734,
17,Sao Filipe,CV,80.78,clear sky,14.8961,-24.4956,
20,Rikitea,PF,73.76,scattered clouds,-23.1203,-134.9692,
21,Ler,SS,77.70,overcast clouds,8.3018,30.1418,
23,Albany,US,74.35,overcast clouds,42.6001,-73.9662,
25,Alta Floresta,BR,85.82,few clouds,-9.8756,-56.0861,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
            

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [15]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cururupu,BR,83.43,few clouds,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
3,Belmonte,BR,77.50,broken clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
10,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Castro,BR,82.65,few clouds,-24.7911,-50.0119,CHACARA BAILLY
16,Coro,VE,89.53,few clouds,11.4045,-69.6734,Federal
17,Sao Filipe,CV,80.78,clear sky,14.8961,-24.4956,Hotel Xaguate
20,Rikitea,PF,73.76,scattered clouds,-23.1203,-134.9692,People ThankYou
21,Ler,SS,77.70,overcast clouds,8.3018,30.1418,
23,Albany,US,74.35,overcast clouds,42.6001,-73.9662,
25,Alta Floresta,BR,85.82,few clouds,-9.8756,-56.0861,Floresta Amazonica Hotel


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cururupu,BR,83.43,few clouds,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
3,Belmonte,BR,77.50,broken clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
10,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Castro,BR,82.65,few clouds,-24.7911,-50.0119,CHACARA BAILLY
16,Coro,VE,89.53,few clouds,11.4045,-69.6734,Federal
17,Sao Filipe,CV,80.78,clear sky,14.8961,-24.4956,Hotel Xaguate
20,Rikitea,PF,73.76,scattered clouds,-23.1203,-134.9692,People ThankYou
25,Alta Floresta,BR,85.82,few clouds,-9.8756,-56.0861,Floresta Amazonica Hotel
30,Georgetown,MY,78.73,scattered clouds,5.4112,100.3354,Cititel Penang
33,Butaritari,KI,83.57,overcast clouds,3.0707,172.7902,Isles Sunset Lodge


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))